In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, LSTM, Bidirectional, SimpleRNN, GRU, Input, Flatten, Multiply
import tensorflow as tf
import glob

# Enable GPU usage
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Load data
files = glob.glob(r'D:\A_NAUSHAD\E\Dataset\Book7.csv')
data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)

# Time column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Feature and target columns
feature_columns = ['Islamabad_PM2.5', 'Dhaka_PM2.5', 'Beijing_PM2.5', 'Delhi_PM2.5']  # Example feature columns
target_columns = feature_columns  # Use feature_columns as target_columns for prediction

def create_model(input_shape):
    models = {}
    
#     # LSTM Model
#     model_LSTM = Sequential()
#     model_LSTM.add(LSTM(16, activation='relu', input_shape=input_shape))
#     model_LSTM.add(Dropout(0.25))
#     model_LSTM.add(Dense(1))
#     model_LSTM.compile(optimizer='adam', loss='mse')
#     models['LSTM'] = model_LSTM
    
#     # GRU Model
#     model_GRU = Sequential()
#     model_GRU.add(GRU(16, activation='tanh', input_shape=input_shape))
#     model_GRU.add(Dropout(0.25))
#     model_GRU.add(Dense(1))
#     model_GRU.compile(optimizer='adam', loss='mse')
#     models['GRU'] = model_GRU
    
#     # RNN Model
#     model_RNN = Sequential()
#     model_RNN.add(SimpleRNN(16, activation='tanh', input_shape=input_shape))
#     model_RNN.add(Dropout(0.25))
#     model_RNN.add(Dense(1))
#     model_RNN.compile(optimizer='adam', loss='mse')
#     models['RNN'] = model_RNN
    
#     # BiLSTM Model
#     model_BiLSTM = Sequential()
#     model_BiLSTM.add(Bidirectional(LSTM(8, activation='tanh'), input_shape=input_shape))
#     model_BiLSTM.add(Dropout(0.25))
#     model_BiLSTM.add(Flatten())
#     model_BiLSTM.add(Dense(1))
#     model_BiLSTM.compile(optimizer='adam', loss='mse')
#     models['BiLSTM'] = model_BiLSTM
    
#     # CNN Model
#     model_CNN = Sequential()
#     model_CNN.add(Conv1D(filters=16, kernel_size=1, input_shape=input_shape))
#     model_CNN.add(Flatten())
#     model_CNN.add(Dropout(0.25))
#     model_CNN.add(Dense(1))
#     model_CNN.compile(optimizer='adam', loss='mse')
#     models['CNN'] = model_CNN
    
#     # Proposed Model 1
#     model_proposed1 = Sequential()
#     model_proposed1.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True), input_shape=input_shape))
#     model_proposed1.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=True)))
#     model_proposed1.add(Bidirectional(LSTM(16, activation='tanh', return_sequences=False)))
#     model_proposed1.add(Dropout(0.1))
#     model_proposed1.add(Dense(1))
#     model_proposed1.compile(optimizer='adam', loss='mse')
#     models['proposed1'] = model_proposed1
    
#     # Proposed Model 2
#     model_proposed2 = Sequential()
#     model_proposed2.add(Conv1D(filters=16, kernel_size=3, padding='causal', activation='relu', input_shape=input_shape))
#     model_proposed2.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True)))
#     model_proposed2.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=True)))
#     model_proposed2.add(Bidirectional(LSTM(16, activation='tanh', return_sequences=False)))
#     model_proposed2.add(Dropout(0.1))
#     model_proposed2.add(Dense(1))
#     model_proposed2.compile(optimizer='adam', loss='mse')
#     models['proposed2'] = model_proposed2
    
#     # Proposed Model 3
#     model_proposed3 = Sequential()
#     model_proposed3.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=input_shape))
#     model_proposed3.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True)))
#     model_proposed3.add(Bidirectional(LSTM(32, activation='tanh', return_sequences=True)))
#     model_proposed3.add(Bidirectional(LSTM(16, activation='tanh', return_sequences=False)))
#     model_proposed3.add(Dropout(0.1))
#     model_proposed3.add(Dense(1))
#     model_proposed3.compile(optimizer='adam', loss='mse')
#     models['proposed3'] = model_proposed3

    # Proposed Model 4 (WaveNet with LSTM)
    def wavenet_layer(inputs, dilation_rate=2, kernel_size=2):
        dilated_conv = Conv1D(filters=32, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')(inputs)
        gated_activation = Activation('tanh')(dilated_conv)
        gated_activation = Multiply()([gated_activation, Activation('sigmoid')(dilated_conv)])
        return gated_activation
    
    inputs = Input(shape=input_shape)
    x = wavenet_layer(inputs)
    x = wavenet_layer(x, dilation_rate=4)
    x = wavenet_layer(x, dilation_rate=8)
    x = Bidirectional(LSTM(64, activation='tanh', return_sequences=True))(x)
    x = Bidirectional(LSTM(32, activation='tanh', return_sequences=True))(x)
    x = Bidirectional(LSTM(16, activation='tanh', return_sequences=False))(x)
    x = Dropout(0.1)(x)
    outputs = Dense(1)(x)
    
    model_proposed4 = Model(inputs=inputs, outputs=outputs)
    model_proposed4.compile(optimizer='adam', loss='mse')
    models['proposed4'] = model_proposed4

    # Proposed Model 5 (WaveNet)
    def WaveNet(input_shape):
        inputs = Input(shape=input_shape)
        x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
        # Add more layers as per your WaveNet architecture
        outputs = Conv1D(filters=1, kernel_size=1, activation='linear')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model

    wavenet_input_shape = (X_train.shape[1], X_train.shape[2]) if len(X_train.shape) == 3 else (X_train.shape[1], 1)
    wavenet_model = WaveNet(wavenet_input_shape)
    models['proposed5'] = wavenet_model

    return models

def evaluate_preds(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mslr = tf.keras.metrics.mean_squared_logarithmic_error(y_true, y_pred)

    results = {
        "mae": mae.numpy().mean(),
        "mse": mse.numpy().mean(),
        "rmse": rmse.numpy().mean(),
        "mape": mape.numpy().mean(),
        "mslr": mslr.numpy().mean(),
    }
    
    for metric, value in results.items():
        print(f"{metric}: {value:.4f}")

    return results

def save_to_csv(df, file_path):
    df.to_csv(file_path, index=False)

def process_and_save_results(model_name, column, df, history, y_test_inverse, y_pred_inverse, model, X_train, X_test, feature_transformed):
    # Forecasting dates
    forecast_dates = pd.date_range(start='2023-01-28', end='2024-12-31', freq='H')
    df_forecast = pd.DataFrame(index=forecast_dates)
    
    # Scale and reshape features
    feature_scaled = feature_transformed.reshape(-1, 1)
    scaler = MinMaxScaler()
    feature_scaled = scaler.fit_transform(feature_scaled)

    forecast_generator = TimeseriesGenerator(feature_scaled, np.zeros(len(feature_scaled)), length=len(X_train), sampling_rate=1, batch_size=1)
    predicted_values_forecast = model.predict(forecast_generator)
    predicted_values_forecast = scaler.inverse_transform(predicted_values_forecast)

    if len(predicted_values_forecast) > len(df_forecast):
        predicted_values_forecast = predicted_values_forecast[:len(df_forecast)]
    else:
        forecast_values = np.full((len(df_forecast), 1), np.nan)
        forecast_values[:len(predicted_values_forecast)] = predicted_values_forecast
        predicted_values_forecast = forecast_values

    df_forecast[column] = predicted_values_forecast
    save_to_csv(df_forecast, f'D:/A_NAUSHAD/E/RESULTS/FORE/{model_name}_{column}_Wave_fore.csv')

    # Plot forecast
    plt.figure(figsize=(12, 6))
    plt.plot(df_forecast.index, df_forecast[column], label='Forecasted')
    plt.xlabel('Hour')
    plt.ylabel(column)
    plt.legend()
    plt.title(f'Forecast of Hourly {column} concentration using {model_name}')
    plt.show()
    
    # Save loss history
    save_to_csv(pd.DataFrame(history.history['loss']), f'D:/A_NAUSHAD/E/RESULTS/LOSS/{model_name}_{column}_Wave_loss.csv')
    save_to_csv(pd.DataFrame(history.history['val_loss']), f'D:/A_NAUSHAD/E/RESULTS/LOSS/{model_name}_{column}_Wave_val_loss.csv')

    # Save training and testing predictions
    predictions_train = model.predict(X_train)
    save_to_csv(pd.DataFrame(predictions_train), f'D:/A_NAUSHAD/E/RESULTS/PRED/{model_name}_{column}_Wave_train_pred.csv')
    
    predictions_test = model.predict(X_test)
    save_to_csv(pd.DataFrame(predictions_test), f'D:/A_NAUSHAD/E/RESULTS/PRED/{model_name}_{column}_Wave_test_pred.csv')

    # Save evaluation results
    eval_results = evaluate_preds(y_true=y_test_inverse, y_pred=y_pred_inverse)
    eval_df = pd.DataFrame.from_dict(eval_results, orient='index', columns=['value'])
    save_to_csv(eval_df, f'D:/A_NAUSHAD/E/RESULTS/EVAL/{model_name}_{column}_Wave_eval.csv')



# Define the callbacks
lr_monitor = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    patience=2,
    factor=0.5,
    cooldown=1,
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Preprocess features and target
for target_column in feature_columns:
    print(f"\nProcessing target column: {target_column}")
    
    # Prepare target data
    target = data[target_column].values
    scaler_target = MinMaxScaler()
    target_scaled = scaler_target.fit_transform(target.reshape(-1, 1))

    # Prepare features
    features = data[feature_columns].values
    scaler_features = MinMaxScaler()
    features_scaled = scaler_features.fit_transform(features)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_scaled, test_size=0.15, random_state=1, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1, shuffle=False)
    
    # Reshape data for LSTM/GRU input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    # Create models
    models = create_model((X_train.shape[1], X_train.shape[2]))
    
    # Train and evaluate each model
    for model_name, model in models.items():
        print(f"\nTraining and evaluating model: {model_name}")
        
        # Train model with callbacks
        history = model.fit(
            X_train, y_train,
            epochs=200,
            batch_size=64,
            validation_data=(X_val, y_val),
            verbose=1,
            callbacks=[lr_monitor, early_stopping]
        )
        
        # Predict and evaluate
        y_pred = model.predict(X_test)
        y_test_inverse = scaler_target.inverse_transform(y_test)
        y_pred_inverse = scaler_target.inverse_transform(y_pred)
        
        # Process and save results
        process_and_save_results(model_name, target_column, data, history, y_test_inverse, y_pred_inverse, model, X_train, X_test, features_scaled)



Processing target column: Islamabad_PM2.5

Training and evaluating model: proposed4
Epoch 1/200
368/368 [==============================] - 22s 25ms/step - loss: 0.0129 - val_loss: 2.7061e-04 - lr: 0.0010
Epoch 2/200
368/368 [==============================] - 6s 16ms/step - loss: 9.0328e-04 - val_loss: 3.9226e-04 - lr: 0.0010
Epoch 3/200
365/368 [============================>.] - ETA: 0s - loss: 7.8964e-04
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
368/368 [==============================] - 6s 16ms/step - loss: 7.8939e-04 - val_loss: 2.7710e-04 - lr: 0.0010
Epoch 4/200
368/368 [==============================] - 6s 16ms/step - loss: 5.9002e-04 - val_loss: 3.3351e-04 - lr: 5.0000e-04
Epoch 5/200
368/368 [==============================] - 6s 16ms/step - loss: 5.9178e-04 - val_loss: 9.8958e-05 - lr: 5.0000e-04
Epoch 6/200
368/368 [==============================] - 6s 16ms/step - loss: 5.5096e-04 - val_loss: 4.5174e-05 - lr: 5.0000e-04
Epoch 7/200
368/368 [=

KeyboardInterrupt: 